# Example usage of Stokes Class

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pybbtd.stokes as stokes
import pybbtd.btd as btd

In [5]:
stokes.hello()

hello again testing


0

In [53]:
# Create Stokes model
R = 3
L = 10
btd.validate_R_L(R, L)
X = stokes.Stokes([100, 100], R, L)

Sufficient condition for uniqueness satisfied
Stokes tensor initialized with dimensions (100, 100, 4) on LL1 mode.


In [54]:
# Create ground truth tensor
np.random.seed(10)
[A0, B0, C0], T0 = X.generate_stokes_tensor()

In [59]:
stokes.load_stokes_tensor(T0, R, L)

Sufficient condition for uniqueness satisfied
Stokes tensor initialized with dimensions (100, 100, 4) on LL1 mode.
All pixels satisfy the Stokes constraints.


In [56]:
A0, B0, C0 = X.factors

In [6]:
X.validate_stokes_tensor()

All pixels satisfy the Stokes constraints.
